## Шаг 1. Откройте файл с данными и изучите общую информацию

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Прочитали файл *data.csv*, сохранили его в переменной *data* и получили общую информацию о данных. Всего в таблице 12 столбцов с типами данных: *object*, *float* и *int*. 

Подробная информация о столбцах: 
* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

В столбцах *days_employed* и *total_income* количество значений отличается от других, значит, в данных имеются пропущенные значения. 

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data_copy = data.copy(deep=True)

**Вывод**

Из таблицы мы видим, что в столбце *days_employed* присутствуют отрицательные значения, что является артефактом и не соответствует действительности. При выгрузке данных могла произойти ошибка, из-за чего значения стали отрицательными. Также в четвертой строке этого столбца можно заметить, что трудовой стаж равен *340266 дням*, что соответствует *932 годам*. Данные ошибки нужно будет отработать и исправить. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

По суммарному количеству пропусков видно, что в некоторых строках пропущена информация о трудовом стаже и ежемесячном доходе. Это можно объяснить тем, что некоторые люди не работают по разным причинам, следовательно не имеют дохода. Данные пропуски необходимо заменить средними значениями по столбцу. Для этого сначала нужно получить абсолютные значения столбца *days_employed*, проверить строки с трудовым стажем, равным сотням лет и заменить их для корректного расчета среднего значения.

In [ ]:
data['days_employed'] = data['days_employed'].abs() #нашли абсолютные значения отрицательного стажа
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
data['dob_years'].max() #максимальный возраст

75

75 лет - максимальный возраст из столбца *dob_years*. Тогда его максимальный рабочий стаж будет равен *250 рабочих дней в год * (75-16) лет = 14750 дней*. Возьмем это значение за максимально возможное в этом столбце. 

In [ ]:
data[data['days_employed'] > 14750]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Проверяем артефакты: 14750 дней - максимально возможный официальный стаж работы (250 рабочих дней в году) с 16 лет до максимального возраста в таблице. Из полученной отфильтрованной таблицы видно, что все эти люди - пенсионеры, поэтому для более аккуратного результата применим корреляцию данных. Найдем отношение медианы работников с огромным стажем к медиане с естественным стажем, затем уменьшим "тысячелетние" стажи на этот коэффициент и таким образом проведем нормирование данных.

In [ ]:
med_over = data[data['days_employed'] > 14750]['days_employed'].median()
med_below = data[data['days_employed'] < 14750]['days_employed'].median()
coef = med_over/med_below
data.loc[data['days_employed'] > 14750,'days_employed'] = data[data['days_employed'] > 14750]['days_employed']/coef
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,1518.332393,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь найдем среднее значение в этом столбце и заменим им пропуски:

In [ ]:
days_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(value=days_employed_avg)
data.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Теперь пропуски остались в столбце *total_income*, заменим их. Так как доход может сильно различаться по столбцу *income_type*, то есть по типу заработка, отработаем пропуски через замену на медиану данного вида заработка с помощью функции transform(), которая в данном случае принимает пустое значение ячейки и заменяет его на медиану данной категории.

In [ ]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(value=x.median()))
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим столбец *children*:

Все пропуски отработаны.

In [ ]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Исправим отрицательные значения "-1", количество элементов незначительно, вероятней всего, это являлось опечаткой и количество относится к элементу "1". Также можно заметить количество детей 20, что смотря на остальные значения, является чем-то неестественным в данной группе,  скорей всего это также является опечаткой и ошибкой при вводе данных, поэтому заменим 20 на 2.

In [ ]:
data.loc[data['children'] == -1,'children'] = 1
data.loc[data['children'] == 20,'children'] = 2
data['children'].value_counts() #сделаем проверку

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

В данном пункте были заменены и проанализированы имеющиеся в столбцах *days_employed* и *total_income* пропуски на среднее значение столбца. После последней проверки видно, что пропусков больше не осталось. Также проверили и отработали столбец *children*, после чего убедились в отсутствии необычных значений.

### Замена типа данных

В данном пункте необходимо провести замену вещественного типа данных на целочисленный. Столбцы *days_employed* и *total_income* имеют тип данных *float* (вещественный), в такой точности нет необходимости, поэтому данные можно преобразовать в тип *int* (целый). Для этого применим метод *astype()*.

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Провели замену типов данных, после проверки видно, что необходимые столбцы теперь имеют тип *int*.

### Обработка дубликатов

In [ ]:
data['education'].value_counts() #посчитали частоту встречаемости уникальных значений

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Можно увидеть, что имеются дубликаты, отличающиеся регистром. Возможная причина - разные менеджера вводили свой вариант значения, из-за чего в итоговом файле они являются дубликатами друг друга. Для исправления этого приведем все строки к нижнему регистру с помощью метода *str.lower()*.

In [ ]:
data['education'] = data['education'].str.lower()
data['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь всё группируется в 5 категорий, и дубликатов нет. Проверим другие столбцы:

In [ ]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Дубликатов нет, все 8 групп встречааются в одинаковом виде.

In [ ]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце *purpose* имеются одинаковые цели, выраженные по-разному. Отработаем этот столбец с помощью лемматизации в следующем пункте.

In [ ]:
data.duplicated().sum()

71

С помощью функций *duplicated() и sum()* видим, что в таблице одинаковых строк - 71. Удалим их с помощью *drop_duplicates()* и снова проверим таблицу на наличие дубликатов:

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.duplicated().sum()

0

**Вывод**

В столбце "образование" были обработы дубликаты, в остальных столбцах значения так же были проверены на дубликаты с помощью метода *value_counts()*. После преобразований проверили, имеются ли в таблице дублирующиеся строки. Нашли 71 дубликат и удалили.

### Лемматизация

Применим лемматизацию для столбца *purpose*.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В данном и предыдущем шаге мы можем видеть уникальные цели по заявкам на кредит, среди них можно выделить основные: операции с жильем, операции по автомобилям, образование, операции с недвижимостью и свадьба. 

В следующем шаге создадим функцию, которая будет лемматизировать все цели, и при нахождении ключевого слова будет выдавать категорию в новом столбце *purpose*. Таким образом, объединим операции с жильем и недвижимостью в единую группу *недвижимость*. Остальные цели сохраним с названиями *свадьба, автомобиль и образование*. Также на случай, если что-то было упущено, добавим опцию *нет категории*.

In [ ]:
from pymystem3 import Mystem
m = Mystem()

def create_category(purp):
    purpose = m.lemmatize(purp)
    
    try:
        if 'свадьба' in purpose:
            return 'свадьба'
        if 'автомобиль' in purpose:
            return 'автомобиль'
        if ('жилье' in purpose) or ('недвижимость' in purpose):
            return 'недвижимость'
        if 'образование' in purpose:
            return 'образование'
    except:
        return 'нет категории'
    
data['purposes'] = data['purpose'].apply(create_category)
data['purposes'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purposes, dtype: int64

**Вывод**

Чтобы лемматизировать столбец *purpose*, для начала получили список уникальных значений из этого столбца методом *unique()*. Затем, лемматизировали каждый элемент из данного списка с помощью написанной функции *create_category*, которая была применена ко всему списку методом *apply()*. Также был создан новый столбец *purposes*, в котором сохранились четыре цели, теперь одинаково сформулированные и которые можно сгруппировать. Также видим, что результат не выдал ничего с опцией *нет категории*, следовательно, всё было сгруппировано верно.

### Категоризация данных

Для дальнейшего анализа необходимо категоризировать данные. В прошлом шаге был категоризирован столбец с целями. Теперь категоризируем столбец с уровнем дохода и добавим новый столбец *children_count*, в котором количество детей будет категоризировано на три группы.

In [ ]:
data['income'] = pd.qcut(data['total_income'],3,labels = ['низкий', 'средний', 'высокий'])
data['income'].value_counts()

средний    7285
низкий     7152
высокий    7017
Name: income, dtype: int64

С помощью функции *qcut* разбили доходы на три равные группы (0.25-, 0.5-, 0.75-квантили): *низкий, средний, высокий*. 

In [ ]:
def child_count(children):
    if children == 0:
        return 'нет детей'
    elif children == 1 or children == 2:
        return '1-2'
    else: return '3+'

data['children_count'] = data['children'].apply(child_count)
data['children_count'].value_counts()

нет детей    14091
1-2           6983
3+             380
Name: children_count, dtype: int64

Теперь для анализа будет удобней использовать столбец *children_count*, чтобы понять, к какой категории касательно количества детей относится клиент.

**Вывод**

Для анализа нам нужны будут категоризированные данные уровня дохода, целей кредита, количества детей, а также семейного положения. Столбец с семейным положением уже был отработан, цели кредита были категоризированы, поэтому в этом пункте был рассмотрен и категоризирован столбец с доходом, а именно поделен на три группы, и добавлен новый столбец *children_id*, где хранятся результаты применения функции *child_id*, оценивающей, имеет ли заемщик детей или нет. Если есть хотя бы один ребенок, функция выдает 1, если нет - 0.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.pivot_table(index=['children_count'],values='debt')

,debt
children_count,
1-2,0.092654
3+,0.081579
нет детей,0.075438


In [ ]:
data.pivot_table(index=['children_count'],values='debt',aggfunc='sum')

,debt
children_count,
1-2,647
3+,31
нет детей,1063


При условии, что детей нет: *P = 1063/14149 = 0.075 = 7.5%*

При условии, что детей в семье 1-2: *P = 647/6996 = 0.092 = 9.2%*

При условии, что детей в семье трое или больше: *P = 31/380 = 0.082 = 8.2%*

**Вывод**

В первой таблице мы видим вероятность существования задолженности при условиях, что у заемщика нет детей, 1-2 детей или 3+ детей. 

Из результата делаем вывод: 
 - вероятность задолженности по сравнению с людьми, у которых нет детей - при наличии одного или двух детей увеличивается на *1.7 процентных пункта до 9.2%*, а при 3+ детей на *0.7 процентных пункта до 8.2%*. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.pivot_table(index=['family_status'],values='debt').sort_values(by='debt',ascending=False)

,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


**Вывод**

В данной сводной таблице отсортированы в порядке убывания вероятности задолженности при различном семейном статусе. Как мы видим, на первом и втором месте статусы *не женат / не замужем* и *гражданский брак* с вероятностью *9.7% и 9.3%*, затем *женат / замужем, в разводе, вдовец / вдова* с вероятностями *7.5%, 7.1% и 6.6%* соотвественно. Значит, вероятность задолженности по возврату кредита выше примерно на *2.5 процентных пункта*, если человек не состоит или не состоял в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data.pivot_table(index=['income'],values='debt').sort_values(by='debt',ascending=False)

,debt
income,
средний,0.086754
низкий,0.081236
высокий,0.075246


**Вывод**

Из данной таблицы видно, что задолженности имеются у всех трех групп, при этом вероятность невозврата кредита в срок минимальна у людей с высоким доходом, затем идут заемщики с низким доходом, и наибольшая вероятность у людей со средним уровнем дохода.
У людей со средним уровнем дохода могут быть совершенно разные причины не погашения кредита в срок, поэтому очень сложно определить зависимость между уровнем дохода и погашением кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data.pivot_table(index=['purposes'],values='debt').sort_values(by='debt',ascending=False)

,debt
purposes,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.072334


**Вывод**

Наибольшую вероятность задолженности имеют люди, взявшие кредит на автомобиль, затем на образование, свадьбу и наименьшую - на недвижимость. 

## Шаг 4. Общий вывод

В проекте был обработан и исследован файл с данными со статистикой о платёжеспособности клиентов. Цель проекта - понять, какие факторы влияют на погашение кредита в срок. Были использованы различные методы обработки данных, группировки, категоризации, чтобы сделать их наиболее "чистыми".

Из проделанного анализа можно сделать вывод о том, какой клиент является наиболее платежеспособным по кредиту и может вернуть его в срок с наибольшей вероятностью. Портрет такого заемщика выглядит так: *человек, не имеющий детей, который состоял или состоит в официальном браке, имеет высокий уровень дохода и цель его кредита - операции с недвижимостью*.